In [1]:
import os
import networkx as nx
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.utils import dense_to_sparse
import torch.optim as optim

In [10]:
"""def read_graph(file_path):
    G = nx.Graph()
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('c') or line.strip() == '':
                continue
            if line.startswith('p'):
                parts = line.split()
                num_vertices = int(parts[2])
                # Add isolated vertices
                G.add_nodes_from(range(num_vertices))
            else:
                u, v = map(int, line.split())
                G.add_edge(u-1, v-1)
    return G"""

def read_graph(file_path):
    G = nx.Graph()
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if line.startswith('c') or line.strip() == '':
                continue
            if i == 0:
                num_vertices = int(line)
                # Add isolated vertices
                G.add_nodes_from(range(num_vertices))
                continue
            if i == 1:
                continue
            else:
                u, v = map(int, line.split())
                G.add_edge(u, v)
    return G

In [3]:
def read_vertex_cover(file_path):
    vertex_cover = set()
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('c') or line.strip() == '':
                continue
            if line.startswith('s'):
                continue
            vertex_cover.add(int(line.strip())-1)
    return vertex_cover

In [4]:
def prepare_data(graph_file, cover_file):
    G = read_graph(graph_file)
    vertex_cover = read_vertex_cover(cover_file)
    
    num_nodes = G.number_of_nodes()
    
    # Adjacency matrix
    adj_matrix = nx.to_numpy_array(G)

    arr = [G.degree(node) for node in G.nodes]
    diag = np.diag(arr)
    node_features = diag

    
    # Labels: 1 if in vertex cover, 0 otherwise
    labels = np.zeros(num_nodes)
    for node in vertex_cover:
        labels[node] = 1
    
    return adj_matrix, node_features, labels

In [17]:
def process_folder(graphs, vc):
    data = {'node_features': [], 'edges': [], 'labels': [], 'graph_id': []}
    
    for filename in os.listdir(graphs):
        if filename.endswith('.mtx'):
            graph_file = os.path.join(graphs, filename)
            cover_file = os.path.join(vc, filename.replace('.mtx', '.mtx.vc'))
            
            if not os.path.exists(cover_file):
                print(f"Warning: No cover file found for {graph_file}")
                continue
            
            # Prepare data
            adj_matrix, node_features, labels = prepare_data(graph_file, cover_file)
            #print(node_features)

            
            # Convert to DataFrames
            df_nodes = pd.DataFrame(node_features)
            #print(df_nodes)
            df_edges = pd.DataFrame([(i, j, adj_matrix[i, j]) for i in range(adj_matrix.shape[0]) for j in range(adj_matrix.shape[1]) if adj_matrix[i, j] > 0],
                                    columns=['source', 'target', 'weight'])
            df_labels = pd.DataFrame(labels, columns=['label'])

            
            # Add a graph identifier
            graph_id = os.path.splitext(filename)[0]
            data['node_features'].append(df_nodes)
            data['edges'].append(df_edges)
            data['labels'].append(df_labels)
            data['graph_id'].append(graph_id)
    
    # Convert lists of DataFrames to single DataFrame with MultiIndex
    #print(data['node_features'])
    df_nodes_all = pd.concat(data['node_features'], keys=data['graph_id'], names=['graph_id', 'node_id'])
    
    df_edges_all = pd.concat(data['edges'], keys=data['graph_id'])
    df_labels_all = pd.concat(data['labels'], keys=data['graph_id'], names=['graph_id', 'node_id'])
    
    return df_nodes_all, df_edges_all, df_labels_all

In [6]:
def convert_to_pyg_data(df_nodes, df_edges, df_labels):
    # Extract node features
    x = torch.tensor(df_nodes.values, dtype=torch.float)
    
    # Extract edges
    edge_index = torch.tensor(df_edges[['source', 'target']].values.T, dtype=torch.long)
    
    # Extract labels
    y = torch.tensor(df_labels.values, dtype=torch.float)
    
    # Create PyTorch Geometric data object
    data = Data(x=x, edge_index=edge_index, y=y)
    return data

In [7]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GCN, self).__init__()
        self.num_layers = num_layers
        
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))
        
    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = F.relu(x)
        x = self.convs[-1](x, edge_index)
        x = torch.sigmoid(x)
        return x

In [8]:
def train_gcn(model, data, epochs=2000, lr=0.001):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()
    
    loader = DataLoader([data], batch_size=64, shuffle=True)
    
    for epoch in range(epochs):
        for batch in loader:
            optimizer.zero_grad()
            out = model(batch.x, batch.edge_index)
            loss = criterion(out.squeeze(), batch.y.squeeze())
                
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
        if epoch % 100 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

In [18]:
df_nodes_all, df_edges_all, df_labels_all = process_folder("./new_graphs/", "./new_results/")
data = convert_to_pyg_data(df_nodes_all, df_edges_all, df_labels_all)
model = GCN(df_nodes_all.shape[1], 64, 1, 20)
train_gcn(model.to('cuda'), data.to('cuda'))

Epoch 1/2000, Loss: 0.6931480169296265
Epoch 101/2000, Loss: 0.6834039092063904
Epoch 201/2000, Loss: 0.6832414865493774
Epoch 301/2000, Loss: 0.6832404732704163
Epoch 401/2000, Loss: 0.6832422018051147
Epoch 501/2000, Loss: 0.6832401156425476
Epoch 601/2000, Loss: 0.6832414269447327
Epoch 701/2000, Loss: 0.6832401156425476
Epoch 801/2000, Loss: 0.6832402944564819
Epoch 901/2000, Loss: 0.6832401156425476
Epoch 1001/2000, Loss: 0.6832656264305115
Epoch 1101/2000, Loss: 0.6832402944564819
Epoch 1201/2000, Loss: 0.6832401156425476
Epoch 1301/2000, Loss: 0.6832401156425476
Epoch 1401/2000, Loss: 0.6832401156425476
Epoch 1501/2000, Loss: 0.6832400560379028
Epoch 1601/2000, Loss: 0.6832400560379028
Epoch 1701/2000, Loss: 0.6832400560379028
Epoch 1801/2000, Loss: 0.6832400560379028
Epoch 1901/2000, Loss: 0.6832400560379028
